In [1]:
import torch
import deepspeed

from transformers import AutoTokenizer, AutoModelForCausalLM
model_path = "/share/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

[2024-10-14 18:53:32,973] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/lzq/miniconda3/envs/react-rl/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/lzq/miniconda3/envs/react-rl/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/lzq/miniconda3/envs/react-rl/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/lzq/miniconda3/envs/react-rl/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/lzq/miniconda3/envs/react-rl/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/lzq/miniconda3/envs/react-rl/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/lzq/minic

In [2]:
from datasets import load_dataset
ds = load_dataset("/home/lzq/react-rl/action-level/action-head/mrpc")

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length", max_length=128)

tokenizer.add_special_tokens({'pad_token': '<PAD>'})
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [3]:
encoded_dataset = ds.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map: 100%|██████████| 1725/1725 [00:00<00:00, 5738.15 examples/s]


In [4]:
ds_json = "/home/lzq/pretrain/ds_config_zero_3.json"
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
model_engine, optimizer, _, _ = deepspeed.initialize(model=model,
                                                     optimizer=optimizer,
                                                     model_parameters=model.parameters(),
                                                     config_params=ds_json)

[2024-10-14 18:53:52,676] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.15.1, git-hash=unknown, git-branch=unknown
[2024-10-14 18:53:52,678] [INFO] [comm.py:652:init_distributed] cdb=None
[2024-10-14 18:53:52,679] [INFO] [comm.py:667:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...


/home/lzq/miniconda3/envs/react-rl/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[2024-10-14 18:53:52,935] [INFO] [comm.py:717:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.10.250.169, master_port=29500
[2024-10-14 18:53:52,936] [INFO] [comm.py:683:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


[W socket.cpp:464] [c10d] The server socket has failed to bind to [::]:29500 (errno: 98 - Address already in use).
[W socket.cpp:464] [c10d] The server socket has failed to bind to 0.0.0.0:29500 (errno: 98 - Address already in use).
[E socket.cpp:500] [c10d] The server socket has failed to listen on any local network address.


DistNetworkError: The server socket has failed to listen on any local network address. The server socket has failed to bind to [::]:29500 (errno: 98 - Address already in use). The server socket has failed to bind to 0.0.0.0:29500 (errno: 98 - Address already in use).

In [7]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(encoded_dataset["train"], batch_size=32, shuffle=True)
print(train_dataloader.dataset)

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 3668
})


In [11]:
epochs = 2
for epoch in range(epochs):
    model_engine.train()
    for step, batch in enumerate(train_dataloader):
        # 将输入数据移动到GPU
        inputs = {key: val.to("cuda:0") for key, val in batch.items() if key != 'label'}

        # 前向传递
        outputs = model_engine(**inputs)
        loss = outputs.loss
        print(outputs)

        # 反向传播和优化
        model_engine.backward(loss)
        model_engine.step()

        if step % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Step {step}, Loss: {loss.item()}")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:2! (when checking argument for argument index in method wrapper_CUDA__index_select)